<a href="https://colab.research.google.com/github/szheng3/recommendation-system/blob/cql/Explore_CQL/DLR2/SA2C_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self-Attentive Sequential Recommender (SASRec) + Supervised Advantage Actor-Critic (SA2C) Recommender on Retail Rocket Data with CQL loss
In this notebook, we train an SASRec-SA2C model to recommend a list of items to users in the dataset. We are using Retail Rocket data to train this model.

1. Clone the git repository containing all the source code

In [1]:

!git clone -b cql https://github.com/szheng3/recommendation-system.git


Cloning into 'recommendation-system'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 115 (delta 26), reused 61 (delta 12), pack-reused 0
Receiving objects: 100% (115/115), 176.63 KiB | 9.30 MiB/s, done.
Resolving deltas: 100% (26/26), done.


2. Install required libraries

In [2]:
#!pip install -r '/content/recommendation-system/Explore_CQL/requirements.txt'

In [3]:
!pip install trfl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 3.7 MB/s eta 0:00:00


3. Download Retail Rocket events log from AWS S3 bucket to local

In [4]:
!wget https://aipi590.s3.amazonaws.com/events.csv -P '/content/recommendation-system/Explore_CQL/Data/RR_data'

--2023-04-29 15:47:43--  https://aipi590.s3.amazonaws.com/events.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 52.217.84.68, 52.217.230.185, 52.216.221.185, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|52.217.84.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94237913 (90M) [text/csv]
Saving to: ‘/content/recommendation-system/Explore_CQL/Data/RR_data/events.csv’

events.csv          100%[===================>]  89.87M  26.1MB/s    in 3.6s    

2023-04-29 15:47:47 (24.9 MB/s) - ‘/content/recommendation-system/Explore_CQL/Data/RR_data/events.csv’ saved [94237913/94237913]



4. Run script to pre-process data and generate replay buffer

In [5]:
!python '/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer.py' --data='/content/recommendation-system/Explore_CQL/Data/RR_data'


2023-04-29 15:47:49.876712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 15:47:51.645229: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Starting to pre-process data...

Sorting and pickling data...

Splitting data into train, validation, and test sets...

Pickling train, validation, and test sets...

Calculating item popularity and storing as dictionary...

Generating replay buffer from train set...

Pickling replay buffer...

Pickling data statistics...

Script completed successfully!


5. Run script to begin training and evaluate model. The model below is SASRec-SA2C. This run is without CQL. 

In [6]:
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v2.py" --model=SASRec --epoch=10 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"

2023-04-29 15:50:08.733187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 15:50:09.770113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using CQL loss
2023-04-29 15:50:12.024710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bool
	 [[{{node Placeholder}}]]
2023-04-29 15:50:12.089920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning 

5b. Run script to beging training and evaluate model. The model below is SASRec-SA2C. This is run with CQL and alpha factor 0.5.

In [7]:

!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v2.py" --model=SASRec --epoch=10 --CQL_alpha=0.5 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"

2023-04-29 18:52:59.657394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 18:53:00.858148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using CQL loss.
2023-04-29 18:53:03.582395: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bool
	 [[{{node Placeholder}}]]
2023-04-29 18:53:03.635065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUM

In [7]:

!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v2.py" --model=SASRec --epoch=10 --CQL_alpha=1.0 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"